<center><h1>Genetic Algorithms Final Project</h1></center>
<center><h1>A genetic algorithm for the project assignment problem</h1></center>
<center><h1>Aalto University, 2017</h1></center>
<center><h1>Elen Vardanyan and Lilit Janjughazyan</h1></center>

In [ ]:
import numpy as np
import random

The basic steps of the developed GA:
<br>
Step 1: Generate a family of initial chromosomes (solutions),
<br>
Step 2: Calculate the fitness of each chromosome,
<br>
Step 3: Select two ‘parent’ chromosomes from the family using binary tournament selection, 
<br>
Step 4: Produce a child chromosome as an o spring from the parents using a crossover operation, 
<br>
Step 5: Allow the child to mutate,
<br>
Step 6: Calculate the fitness of the child, 
<br>
Step 7: Replace the least fit member of the family with the child providing that the child is fitter
and is not already a member of the family,
<br>
Step 8: Stop if the number of predefined cycles has been reached, otherwise go to step 3.

In [ ]:
SP_matrix           # GLOBAL, an sxp matrix of preferences
priority_weights    # GLOBAL, an s-dimensional list

## 1. The initial family of chromosomes

In [ ]:
def generate_chromosomes(SP_matrix):
    
    """
    param SP_matrix: the student-project matrix (a two-dimensional numpy array)
    return: a list of lists, each of which is a chromosome
    """    
    
    rows = [] # students
    cols = [] # projects

    # remove assignments that can be preset
    for p in range(SP_matrix.shape[1]):
        if 1 in np.unique(SP_matrix[:,p]):
            rows.append(list(SP_matrix[:, p]).index(1))
            cols.append(p)
    np.delete(SP_matrix, rows, 0)
    np.delete(SP_matrix, cols, 1)
        
    N = SP_matrix.shape[0]
    chromosomes = [random.sample(range(SP_matrix.shape[1]), SP_matrix.shape[0]) for chromosome in range(N)]
    
    return chromosomes

## 2. The fitness function

In [ ]:
def fitness_function(chromosome, B=100, M=10, h=lambda x: x**2):
    
    """
    param chromosome: a list of genes/projects, i-th element of which is the project allocated to the i-th student
    param B: a penalty, a suitably large value assigned to the C_ir_ig coefficient if a project g is outside of the student's preferences
    param M: a penalty, a suitably large value assigned to the C_ir_ig coefficient if a project g is assigned to more than one student
    param h: a lambda function, preferably a non-linear function on the student's preferences
    return fitness: the fitness of the chromosome
    """
    
    c = np.zeros(len(chromosome))
    for i in range(len(chromosome)):
        if chromosome[i] not in SP_matrix[i, :]:
            c[i] = B
        for j in range(len(chromosome)):
            if chromosome[i] == chromosome[j] & i!=j:
                c[i] = c[j] = M
        else:
            c[i] = SP_matrix[i, chromosome[i]]
                
    fitness = sum([h(c[i])*priority_weights[i] for i in range(len(chromosome))]) 
    
    return fitness

## 3. Parent selection using a binary tournament

In [ ]:
def binary_tournament(chromosomes):
    
    """
    param chromosomes: a list of lists, each of which is a chromosome
    return parent1, parent2: lists, the parents chosen to reproduce the child in the next step
    """
    
    parent1, parent2 = random.sample(chromosomes, 2)
    if fitness_function(parent1) > fitness_function(parent2):
        parent1 = parent2
        
    parent3, parent4 = random.sample(chromosomes, 2)
    if fitness_function(parent3) < fitness_function(parent4):
        parent2 = parent3
    else:
        parent2 = parent4
    
    return parent1, parent2

## 4. Producing an offspring with a crossover operation

In [ ]:
def crossover(u, v):
    
    """
    param u, v: lists, parents that have to form a child
    return c: a list, the child produced from the crossover
    """
    
    f_u = fitness_function(u)
    f_v = fitness_function(v)
    c = np.zeros(len(u), dtype=int)
    if (f_u == 0) & (f_v == 0):
        prob_u = prob_v = 0.5
    else:
        prob_u = f_v/(f_u+f_v)
        prob_v = f_u/(f_u+f_v)
        
        
    c[0] = np.random.choice([u[0],v[0]], p=[prob_u, prob_v])
    
    for i in range(1,len(u)):
        if (u[i] not in c[:i]) & (v[i] not in c[:i]):
            if u[i] == v[i]:
                c[i] = u[i] = v[i]
            else:
                c[i] = np.random.choice([u[i],v[i]], p=[prob_u, prob_v])  
        elif (u[i] in c[:i]) & (v[i] in c[:i]):
            c[i] = np.random.choice(list(set(range(SP_matrix.shape[1]))-set(c[:i])))
        elif (u[i] in c[:i]) & (v[i] not in c[:i]):
                c[i]=v[i]
        elif (v[i] in c[:i]) & (u[i] not in c[:i]):
                c[i]=u[i] 
                
    return c

## 5. Mutation

In [ ]:
def mutate(chromosome, prob=None):
    
    """
    param chromosome: a list of genes/projects, i-th element of which is the project allocated to the i-th student
    param prob: a float between (0,1), the probability of mutation, default is None (makes the probability )
    """
    
    for i in range(len(chromosome)):
        p = random.uniform(0, 1)
        if p < (prob == None) * 1/(SP_matrix.shape[0] * np.sqrt(len(chromosome))) + (prob != None) * prob:
            if SP_matrix.shape[0] == SP_matrix.shape[1]: # in case when the num of students == the num of projects, do the mutation by swapping
                chromosome[i], chromosome[len(chromosome)-i] = chromosome[len(chromosome)-i], chromosome[i] 
            else: # otherwise, choose the new gene among the projects not already in the chromosome
                chromosome[i] = np.random.choice(list(set(range(SP_matrix.shape[1])) - set(chromosome)))

## 6. Replacement

In [ ]:
def replace(child, chromosomes):
    
    """
    param child: a list, the newest child which may replace the least fit chromosome 
    param chromosomes: a list of lists, each of which is a chromosome
    """
    
    least_fit_index = 0
    least_fitness = fitness_function(chromosomes[0])
    for i in range(1, len(chromosomes)):
        if least_fitness < fitness_function(chromosomes[i]):
            least_fitness = fitness_function(chromosomes[i])
            least_fit = chromosomes[i]
            least_fit_index = i
    chromosomes[least_fit_index] = child # checked if the child is already in chromosomes or not in GA(num_of_cycles)

## The Main Algorithm 

In [ ]:
def GA(num_of_cycles):
    chromosomes = generate_chromosomes(SP_matrix)
    pop_size = len(chromosomes)
    for i in range(num_of_cycles):
        fitnesses = []
        for j in range(len(chromosomes)):
            fitnesses.append(fitness_function(chromosomes[j]))

        parent1, parent2 = binary_tournament(chromosomes)
        child = crossover(parent1, parent2)
        mutate(child, 0.01)
        child = list(child) # to avoid problems of comparisons with lists
        child_fitness = fitness_function(child)
        if (child not in chromosomes) & (child_fitness < min(fitnesses)):
            replace(child, chromosomes)
        fitnesses = [] # update fitnesses after possible replacement
        for j in range(len(chromosomes)):
            fitnesses.append(fitness_function(chromosomes[j]))

        print("Fittest in the {}-th cycle:   \t\t\t".format(i), chromosomes[fitnesses.index(min(fitnesses))], "Fitness: ", min(fitnesses))
        print("Least Fit in the {}-th cycle: \t\t\t".format(i), chromosomes[fitnesses.index(max(fitnesses))], "Fitness: ", max(fitnesses), "\n")
    print("Family of chromosomes after {} cycles \n".format(num_of_cycles))
    for chromosome in chromosomes:
        print("{}) ".format(chromosomes.index(chromosome)+1), *chromosome, " with fitness ", fitness_function(chromosome))